In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

#import math tools
import numpy as np
from scipy.linalg import expm

# We import the tools to handle general Graphs
import networkx as nx

# We import plotting tools 
import matplotlib.pyplot as plt 
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

import account

# Loading your IBM Q account(s)
IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-university', group='harvard-lukin', project='phys160')
IBMQ.get_provider(hub='ibm-q-university', group='harvard-lukin', project='phys160')

configrc.store_credentials:WARNING:2020-05-03 10:48:10,416: Credentials already present. Set overwrite=True to overwrite.


<AccountProvider for IBMQ(hub='ibm-q-university', group='harvard-lukin', project='phys160')>

In [2]:
# encode initial state, returns a quantum circuit
def encode(initial_state):
    qr = QuantumRegister(5) # 5 lines, 3 are for encoding (0,1,2), 2 are ancillary qubits (3,4)
    cr = ClassicalRegister(2) # 2 classical registers for measurement outcomes
    qc = QuantumCircuit(qr, cr)
    qc.initialize(initial_state, 0)
    qc.cx(qr[0],qr[1])
    qc.cx(qr[0],qr[2])
    return qc

In [ ]:
# after performing an operation on the encoded state, correct the error
def correct_error(qc):
    qr = qc.qregs[0]
    cr = qc.cregs[0]
    
    # initialize the ancillary qubits to 0
    qc.initialize([1,0], 3)
    qc.initialize([1,0], 4)
    
    # attach CNOTs
    qc.cx(qr[0], qr[3])
    qc.cx(qr[1], qr[3])
    
    qc.cx(qr[0], qr[4])
    qc.cx(qr[2], qr[4])
    
    # measure the ancillary qubits
    qc.measure(qr[3], cr[0]) # first ancillary qubit's measurement in first classical register
    qc.measure(qr[4], cr[1]) # second ancillary qubit's measurement in second classical register
    
    # apply X gate to correct error based on measurement outcomes
    qc.x(qr[0]).c_if(cr, 3) # if the classical register says |11>, or "3", then apply on qb 0
    qc.x(qr[1]).c_if(cr, 2) # |10> means qb 2 has the error
    qc.x(qr[2]).c_if(cr, 1) # |01> means qb 3 has the error
    
    #remove the gates that were just added
    for i in range(9):
        qc.data.pop(len(qc.data)-1)